In [1]:
import sys
from pathlib import Path
srcdir = Path("../..").resolve()
print(f"Adding {srcdir} to sys.path, this is necessary to import from src")
sys.path.insert(0, str(srcdir))
print(sys.path)

from src.data import data_tools
from pathlib import Path
import gin
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from src.models import rnn_models, train_model
from torch import optim


Adding /Users/rgrouls/code/ML22 to sys.path, this is necessary to import from src
['/Users/rgrouls/code/ML22', '/Users/rgrouls/code/ML22/notebooks/3_recurrent_networks', '/Users/rgrouls/.pyenv/versions/3.9.16/lib/python39.zip', '/Users/rgrouls/.pyenv/versions/3.9.16/lib/python3.9', '/Users/rgrouls/.pyenv/versions/3.9.16/lib/python3.9/lib-dynload', '', '/Users/rgrouls/Library/Caches/pypoetry/virtualenvs/deep-learning-HUU8cknU-py3.9/lib/python3.9/site-packages']


# 1 Iterators
We will be using an interesting dataset. [link](https://tev.fbk.eu/resources/smartwatch)

From the site:
> The SmartWatch Gestures Dataset has been collected to evaluate several gesture recognition algorithms for interacting with mobile applications using arm gestures. Eight different users performed twenty repetitions of twenty different gestures, for a total of 3200 sequences. Each sequence contains acceleration data from the 3-axis accelerometer of a first generation Sony SmartWatch™, as well as timestamps from the different clock sources available on an Android device. The smartwatch was worn on the user's right wrist. 


In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32)
train = streamers["train"]
valid = streamers["valid"]

2023-06-06 11:32:11.684 | INFO     | mads_datasets.datasetfactory:download_data:94 - Dataset already exists at /Users/rgrouls/.cache/mads_datasets/gestures
2023-06-06 11:32:11.692 | INFO     | mads_datasets.datasetfactory:download_data:104 - Digest of downloaded /Users/rgrouls/.cache/mads_datasets/gestures/gestures-dataset.zip matches expected digest
100%|██████████| 651/651 [00:00<00:00, 921.01it/s]


In [3]:
len(train), len(valid)

(81, 20)

In [4]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y

(torch.Size([32, 51, 3]),
 tensor([17,  0,  7,  9,  5,  5,  9, 13, 18, 14,  7, 14, 17,  5, 11,  6, 19,  5,
          6,  4, 19,  0, 12, 14,  4, 19, 17, 17,  2,  0,  6, 10]))

Can you make sense of the shape?
What does it mean that the shapes are sometimes (32, 27, 3), but a second time might look like (32, 30, 3)? In other words, the second (or first, if you insist on starting at 0) dimension changes. Why is that? How does the model handle this? Do you think this is already padded, or still has to be padded?


# 2 Excercises
Lets test a basemodel, and try to improve upon that.

Fill the gestures.gin file with relevant settings for `input_size`, `hidden_size`, `num_layers` and `horizon` (which, in our case, will be the number of classes...)

As a rule of thumbs: start lower than you expect to need!

In [5]:
from src.settings import TrainerSettings, ReportTypes
from src.models.metrics import Accuracy

accuracy = Accuracy()

settings = TrainerSettings(
    epochs=10,
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.GIN, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs=None
)
settings

epochs: 10
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.GIN: 1>, <ReportTypes.TENSORBOARD: 2>, <ReportTypes.MLFLOW: 3>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: None

In [6]:
gin.parse_config_file("gestures.gin")
model = rnn_models.BaseRNN()

In [7]:
gin.get_bindings("BaseRNN")

{'input_size': 3, 'hidden_size': 128, 'num_layers': 3, 'horizon': 20}

Test the model. What is the output shape you need? Remember, we are doing classification!

In [8]:
yhat = model(x)
yhat.shape

torch.Size([32, 20])

Test the accuracy

In [9]:
accuracy(y, yhat)

tensor(0.)

What do you think of the accuracy? What would you expect from blind guessing?

Check shape of `y` and `yhat`

In [10]:
yhat.shape, y.shape

(torch.Size([32, 20]), torch.Size([32]))

And look at the output of yhat

In [11]:
yhat[0]

tensor([ 0.0121, -0.0644, -0.0528, -0.0295, -0.0008,  0.0250,  0.0220, -0.1761,
        -0.0566, -0.0998, -0.0546,  0.0126, -0.1668, -0.0077, -0.0334,  0.0628,
        -0.0624, -0.0496, -0.1477, -0.0023], grad_fn=<SelectBackward0>)

Does this make sense to you? If you are unclear, go back to the classification problem with the MNIST, where we had 10 classes.

We have a classification problem, so we need Cross Entropy Loss.
Remember, [this has a softmax built in](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) 

In [12]:
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(yhat, y)
loss

tensor(2.9951, grad_fn=<NllLossBackward0>)

In [14]:
gin.get_bindings("BaseRNN")

{'input_size': 3, 'hidden_size': 128, 'num_layers': 3, 'horizon': 20}

In [16]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("../../models/gestures/").resolve()
gin.parse_config_file("gestures.gin")

with mlflow.start_run():
    mlflow.set_tag("model", "GRUmodel")
    mlflow.set_tag("dev", "raoul")
    mlflow.log_params(gin.get_bindings("BaseRNN"))

    model = rnn_models.BaseRNN()
    trainer = train_model.Trainer(
        model=model, 
        settings=settings, 
        loss_fn=loss_fn,
        optimizer=optim.Adam, 
        traindataloader=trainstreamer, 
        validdataloader=validstreamer, 
        scheduler=optim.lr_scheduler.ReduceLROnPlateau
    )
    trainer.loop()

    tag = datetime.now().strftime("%Y%m%d-%H%M")
    modelpath = modeldir / (tag + "model.pt")
    torch.save(model, modelpath)

2023-06-06 11:34:05.504 | INFO     | src.data.data_tools:dir_add_timestamp:146 - Logging to gestures/20230606-1134
100%|██████████| 81/81 [00:01<00:00, 53.30it/s]
2023-06-06 11:34:07.174 | INFO     | src.models.train_model:report:210 - Epoch 0 train 2.6675 test 2.4786 metric ['0.1203']
100%|██████████| 81/81 [00:01<00:00, 61.28it/s]
2023-06-06 11:34:08.639 | INFO     | src.models.train_model:report:210 - Epoch 1 train 2.4253 test 2.4161 metric ['0.1250']
100%|██████████| 81/81 [00:01<00:00, 55.25it/s]
2023-06-06 11:34:10.250 | INFO     | src.models.train_model:report:210 - Epoch 2 train 2.3768 test 2.4860 metric ['0.1391']
100%|██████████| 81/81 [00:01<00:00, 60.56it/s]
2023-06-06 11:34:11.723 | INFO     | src.models.train_model:report:210 - Epoch 3 train 2.3681 test 2.3243 metric ['0.1594']
100%|██████████| 81/81 [00:01<00:00, 57.42it/s]
2023-06-06 11:34:13.313 | INFO     | src.models.train_model:report:210 - Epoch 4 train 2.3046 test 2.2737 metric ['0.1609']
100%|██████████| 81/81 [0

Try to update the code above with the following two commands.
    
```python
gin.parse_config_file('gestures.gin')
model = rnn_model.RNNModel()
```

To discern between the changes, also modify the tag mlflow.set_tag("model", "new-tag-here") where you add
a new tag of your choice. This way you can keep the models apart.

In [ ]:
mlflow.end_run()

Excercises:

- improve the RNN model
- test different things. What works? What does not?
- experiment with either GRU or LSTM layers, create your own models + ginfiles. 
- experiment with adding Conv1D layers.

You should be able to get above 90% accuracy with the dataset.